In [13]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import PolynomialFeatures

from util.eval_model import eval_metrics

%matplotlib inline

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [19]:
iris = load_iris()
X = pd.DataFrame(data=iris.data, columns=iris.feature_names)
y = pd.DataFrame(data=iris.target, columns=['CLASS'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
pipeline = Pipeline(
    [
        ('poly', PolynomialFeatures()),
        ('regr', LogisticRegression())
    ]
)

parameters = [
    {
        'poly__degree': [2, 3],
        'regr__C': np.arange(1, 10),
        'regr__penalty': ['l2'],
        'regr__solver': ['newton-cg', 'lbfgs', 'sag']
    },
    {
        'poly__degree': [2, 3],
        'regr__C': np.arange(1, 10),
        'regr__penalty': ['l1'],
        'regr__solver': ['liblinear', 'saga']
    }
]

grid = GridSearchCV(pipeline, param_grid=parameters, cv=10, n_jobs=-1)
grid.fit(X_train, y_train.values.ravel())
y_predict = grid.predict(X_test)

test_score = grid.score(X_test, y_test)

print(grid.best_score_)
print(grid.best_params_)
print(test_score)


0.96
{'poly__degree': 2, 'regr__C': 1, 'regr__penalty': 'l2', 'regr__solver': 'sag'}
0.98


In [26]:
test1 = make_pipeline(PolynomialFeatures(degree=2), LogisticRegression(penalty='l2', solver='sag', C=1))
test1.fit(X_train, y_train.values.ravel())
print(f'Test1 score (L1, sag):\t{test1.score(X_test, y_test.values.ravel())}')

test2 = make_pipeline(PolynomialFeatures(degree=2), LogisticRegression(penalty='l1', solver='saga', C=1))
test2.fit(X_train, y_train.values.ravel())
print(f'Test2 score (L1, saga):\t{test2.score(X_test, y_test.values.ravel())}')

Test1 score (L1, sag):	0.98
Test2 score (L1, saga):	0.98
